In [ ]:
import torch
import torch.nn as nn
from models.layers import Embedding, VisualProjection, FeatureEncoder, CQAttention, CQConcatenate, Conv1D, SeqPANPredictor
from models.layers import DualAttentionBlock
import torch.nn.functional as F
import numpy as np

class SeqPAN(nn.Module):
    def __init__(self, configs, word_vectors):
        super(SeqPAN, self).__init__()
        self.configs = configs
        dim = configs.model.dim
        droprate = configs.model.droprate

        max_pos_len = self.configs.max_pos_len
        self.text_encoder = Embedding(num_words=configs.num_words, num_chars=configs.num_chars, out_dim=dim,
                                       word_dim=configs.model.word_dim, 
                                       char_dim=configs.model.char_dim, 
                                       word_vectors=word_vectors,
                                       droprate=droprate)
                                       
        self.video_affine = VisualProjection(visual_dim=configs.model.video_feature_dim, dim=dim,
                                             droprate=droprate)
        # self.feat_encoder = FeatureEncoder(dim=dim, kernel_size=7, num_layers=4,
        #                                       max_pos_len=max_pos_len, droprate=droprate)


        # self.dual_attention_block_1 = DualAttentionBlock(configs=configs, dim=dim, num_heads=configs.model.num_heads, 
        #                                                 droprate=droprate, use_bias=True, activation=None)
        # self.dual_attention_block_2 = DualAttentionBlock(configs=configs, dim=dim, num_heads=configs.model.num_heads, 
        #                                             droprate=droprate, use_bias=True, activation=None)



        # # self.tfeat_encoder = FeatureEncoder(dim=dim, num_heads=configs.model.num_heads, kernel_size=7, num_layers=4,
        # #                                       max_pos_len=max_pos_len, droprate=droprate)

        # self.q2v_attn = CQAttention(dim=dim, droprate=droprate)
        # self.v2q_attn = CQAttention(dim=dim, droprate=droprate)
        # self.cq_cat = CQConcatenate(dim=dim)
        # self.match_conv1d = Conv1D(in_dim=dim, out_dim=4)

        # lable_emb = torch.empty(size=[dim, 4], dtype=torch.float32)
        # lable_emb = torch.nn.init.orthogonal_(lable_emb.data)
        # self.label_embs = nn.Parameter(lable_emb, requires_grad=True)
        
        # self.predictor = SeqPANPredictor(configs)



    def forward(self, word_ids, char_ids, vfeat_in, vmask, tmask):
        B = vmask.shape[0]
        # tfeat = self.text_encoder(word_ids, char_ids)
        vfeat = self.video_affine(vfeat_in)
        return vfeat
        # vfeat = self.feat_encoder(vfeat)
        # tfeat = self.feat_encoder(tfeat)

        # vfeat_ = self.dual_attention_block_1(vfeat, tfeat, vmask, tmask)
        # tfeat_ = self.dual_attention_block_1(tfeat, vfeat, tmask, vmask)
        # vfeat = vfeat_
        # tfeat = tfeat_

        # vfeat_ = self.dual_attention_block_2(vfeat, tfeat, vmask, tmask)
        # tfeat_ = self.dual_attention_block_2(tfeat, vfeat, tmask, vmask)

        # vfeat = vfeat_
        # tfeat = tfeat_


        # t2v_feat = self.q2v_attn(vfeat, tfeat, vmask, tmask)
        # v2t_feat = self.v2q_attn(tfeat, vfeat, tmask, vmask)
        
        # fuse_feat = self.cq_cat(t2v_feat, v2t_feat, tmask)

        # match_logits = self.match_conv1d(fuse_feat)


        # match_score = F.gumbel_softmax(match_logits, tau=0.3)
        # match_probs =torch.log(match_score)



        # soft_label_embs = torch.matmul(match_score, torch.tile(self.label_embs, (B, 1, 1)).permute(0, 2, 1))

        # fuse_feat = (fuse_feat + soft_label_embs) * vmask.unsqueeze(2)
        # start_logits, end_logits = self.predictor(fuse_feat, vmask)
        # return start_logits, end_logits, match_probs, self.label_embs

In [4]:
from torchsummary import summary
# from tensorboardX import SummaryWriter

import torch
import tensorwatch as tw
from easydict import EasyDict
from utils.data_gen import load_dataset
from utils.utils import load_json
configs = EasyDict(load_json("config/charades/main_c3dFT.json"))
configs.suffix = ""
dataset = load_dataset(configs)
configs.num_chars = dataset['n_chars']
configs.num_words = dataset['n_words']
configs.train.batch_size = 1

model=SeqPAN(configs, dataset['word_vector'])
summary(model, [[1, 11], [1, 11, 12], [1, 64, 1024], [1, 64], [1, 11]], dtypes=[torch.long, torch.long, torch.float, torch.float, torch.float])
# '''tensorboardX生成日志文件'''
# dummy_input01 = torch.rand(10, 1, 28, 28)  # 假设输入10张1*28*28的图片
# dummy_input02 = torch.rand(10, 1, 28, 28)  # 假设输入10张1*28*28的图片
# with SummaryWriter(comment='Net') as w:
#     w.add_graph(Net, (dummy_input01,dummy_input02))


RuntimeError: Failed to run torchsummary. See above stack traces for more details. Executed layers up to: [WordEmbedding: 2-1, Dropout: 3-1, Embedding: 3-2, Dropout: 3-3]

In [ ]:
word_ids = torch.rand([1, 11])
char_ids = torch.rand([1, 11, 12]).long()
vfeats = torch.rand( [1, 64, 1024]).float()
vmask = torch.rand( [1, 64]).float()
tmask = torch.rand( [1, 11]).float()

In [ ]:
model(word_ids, char_ids, vfeats, vmask, tmask)

In [5]:
from models.model import SeqPAN
model=SeqPAN(configs, dataset['word_vector'])
# print(model)|

In [6]:
for name,parameters in model.named_parameters():
    print(name,':',parameters.size())


label_embs : torch.Size([128, 4])
text_encoder.word_emb.pad_vec : torch.Size([1, 300])
text_encoder.word_emb.unk_vec : torch.Size([1, 300])
text_encoder.word_emb.glove_vec : torch.Size([1269, 300])
text_encoder.char_emb.char_emb.weight : torch.Size([36, 100])
text_encoder.char_emb.char_convs.0.0.weight : torch.Size([10, 100, 1, 1])
text_encoder.char_emb.char_convs.0.0.bias : torch.Size([10])
text_encoder.char_emb.char_convs.1.0.weight : torch.Size([20, 100, 1, 2])
text_encoder.char_emb.char_convs.1.0.bias : torch.Size([20])
text_encoder.char_emb.char_convs.2.0.weight : torch.Size([30, 100, 1, 3])
text_encoder.char_emb.char_convs.2.0.bias : torch.Size([30])
text_encoder.char_emb.char_convs.3.0.weight : torch.Size([40, 100, 1, 4])
text_encoder.char_emb.char_convs.3.0.bias : torch.Size([40])
text_encoder.query_conv1d.conv1d.weight : torch.Size([128, 400, 1])
text_encoder.query_conv1d.conv1d.bias : torch.Size([128])
text_encoder.q_layer_norm.weight : torch.Size([128])
text_encoder.q_layer_